In [118]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from python_module.pricing_model import BlackScholesModel
pd.options.display.float_format = '{:.2f}'.format

In [125]:
# -------
# INPUT
# -------
target_delta = -0.2
time_to_maturity = 5

In [126]:
# Import data
df = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)['Adj Close'].to_frame('price')

# Compute volatility estimate
df['realized_volatility'] = np.sqrt(np.log(df['price']/df['price'].shift(1)).pow(2).ewm(span=20).mean()*250)
df = df.dropna()

# Reshape data lenght 
df = df.iloc[:int(df.shape[0] / time_to_maturity) * time_to_maturity]
df['day_to_maturity'] = list(range(time_to_maturity, 0, -1)) * int(df.shape[0] / time_to_maturity)

# Compute strike
for index in df.index:
    if df.loc[index, 'day_to_maturity'] == 5:
        
        spot = df.loc[index, 'price']
        
        atm_delta = BlackScholesModel.compute_option(S=spot, K=spot, T=5/250, r=0, sigma=0.2, option_type='put', compute_greeks=True)['delta']
        delta_0 = BlackScholesModel.compute_option(S=spot, K=spot-1, T=5/250, r=0, sigma=0.2, option_type='put', compute_greeks=True)['delta']
        delta_1 = BlackScholesModel.compute_option(S=spot, K=spot+1, T=5/250, r=0, sigma=0.2, option_type='put', compute_greeks=True)['delta']
        
        ddelta = delta_1-delta_0
        
        dk = 2

        slope = ddelta/dk

        df.loc[index, 'K'] = spot + ((target_delta-atm_delta)/slope)
df = df.ffill()

In [127]:
for index in df.index:

    spot = df.loc[index, 'price']
    
    iv = df.loc[index, 'realized_volatility']

    T = df.loc[index, 'day_to_maturity'] / 250

    K = df.loc[index, 'K']

    results = BlackScholesModel.compute_option(S=spot, K=K, T=T, r=0, sigma=iv, option_type='put', compute_greeks=True)

    for key in results:
        df.loc[index, key] = results[key]
        

In [128]:
df.head(20)

,price,realized_volatility,day_to_maturity,K,delta,gamma,vega,theta,vanna,volga
Date,,,,,,,,,,
1993-02-01,0.01,0.11,5,24.16,-0.04,0.20,0.00,-0.00,-1.25,7.94
1993-02-02,0.00,0.08,4,24.16,-0.00,0.02,0.00,-0.00,-0.17,1.59
1993-02-03,0.00,0.12,3,24.16,-0.00,0.01,0.00,-0.00,-0.08,0.65
1993-02-04,0.00,0.11,2,24.16,-0.00,0.00,0.00,-0.00,-0.00,0.00
1993-02-05,0.00,0.09,1,24.16,-0.00,0.00,0.00,-0.00,-0.00,0.00
1993-02-08,0.00,0.08,5,24.56,-0.01,0.07,0.00,-0.00,-0.63,5.42
1993-02-09,0.00,0.09,4,24.56,-0.03,0.23,0.00,-0.00,-1.40,8.44
1993-02-10,0.00,0.08,3,24.56,-0.01,0.07,0.00,-0.00,-0.48,3.40
1993-02-11,0.00,0.08,2,24.56,-0.00,0.00,0.00,-0.00,-0.01,0.11
